# dual plot troubleshooting
__objective__: get 2x separate plots in a single interactive visualization for saturation binding isotherm and scatchard

In [ ]:
# import libraries
import os
import sys
import numpy as np
import matplotlib.pyplot as plt

from bokeh.layouts import row, column
from bokeh.models import CustomJS, Slider, Label, Span
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import output_notebook

root_dir = os.path.join(os.getcwd(), '..')
sys.path.append(root_dir)

from pharmaplot import mm
import pharmaplot.receptors as rec

In [ ]:
# set to display in notebook as opposed to making an html
output_notebook()

# troubleshoot underlying equations
__references__:
* Hulme, Edward C., and Mike A. Trevethick. "Ligand binding assays at equilibrium: validation and interpretation." British journal of pharmacology 161.6 (2010): 1219-1237. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3000649/
* "Cell Surface Receptors" LE Limbird Chapter 3 - Radioligand Binding
* https://www.graphpad.com/guides/prism/7/curve-fitting/reg_specific_hill.htm
* Attie, Alan D., and Ronald T. Raines. "Analysis of receptor-ligand interactions." Journal of chemical education 72.2 (1995): 119. https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5521016/
* http://www.pdg.cnb.uam.es/cursos/Barcelona2002/pages/Farmac/Comput_Lab/Guia_Glaxo/chap3b.html

### specific binding
note: use javascript <code>Math.log10()</code> to convert to log within the script

In [ ]:
def specific_binding(l, bmax, kd):
    return (l*bmax)/(l + kd)

log_start = -9
log_end = -3
bmax= 100
kd = 1e-6

x_line = np.logspace(log_start, log_end, num=100)
y_line = specific_binding(x_line, bmax, kd)

x_points = np.logspace(log_start, log_end, num=16)
y_points = specific_binding(x_points, bmax, kd)

plt.plot(x_line, y_line)
plt.scatter(x_points, y_points)
plt.show()

plt.plot(x_line, y_line)
plt.scatter(x_points, y_points)
plt.semilogx()
plt.show()

### scatchard

In [ ]:
def scatchard(l, bmax, kd):
    b = specific_binding(l, bmax, kd)
    b_f = b/l
    return b, b_f

log_start = -9
log_end = -3
bmax= 100
kd = 1e-6

x_line = np.logspace(log_start, log_end, num=2)
b_line, bf_line = scatchard(x_line, bmax, kd)
b_points, bf_points = scatchard(np.logspace(log_start, log_end, num=16), bmax, kd)

plt.plot(b_line, bf_line)
plt.scatter(b_points, bf_points)
plt.plot(*scatchard(x_line, 50, 1e-6))
plt.plot(*scatchard(x_line, 100, 3e-6))
# do scatchard B/F transformation within the equation? possible in JS?

### specific binding with hill slope

In [ ]:
def specific_binding_hill(l, bmax, kd, hill_coef):
    return ((l**hill_coef)*bmax)/((l**hill_coef) + (kd**hill_coef))

log_start = -9
log_end = -3
bmax= 100
kd = 1e-6
hill = 3

x_line = np.logspace(log_start, log_end, num=100)
y_line = specific_binding_hill(x_line, bmax, kd, hill)
y_hill_1 = specific_binding(x_line, bmax, kd)

x_points = np.logspace(log_start, log_end, num=16)
y_points = specific_binding_hill(x_points, bmax, kd, hill)


plt.plot(x_line, y_line)
plt.scatter(x_points, y_points)
plt.plot(x_line, y_hill_1, color='orange')
plt.semilogx()
plt.show()

In [ ]:
log_start = -9
log_end = -3
bmax= 100
kd = 1e-6
hill = 3

x_line = np.logspace(log_start, log_end, num=100)
x_log_line = np.log10(x_line)
y_line = specific_binding_hill(x_line, bmax, kd, hill)
y_hill_1 = specific_binding(x_line, bmax, kd)

x_points = np.logspace(log_start, log_end, num=16)
x_log_points = np.log10(x_points)
y_points = specific_binding_hill(x_points, bmax, kd, hill)


plt.plot(x_log_line, y_line)
plt.scatter(x_log_points, y_points)
plt.plot(x_log_line, y_hill_1, color='orange')
plt.show()

## troubleshoot specific binding

In [ ]:
## generate bokeh plot using the following data
log_start = -9
log_end = -3
bmax= 100
kd = 1e-6

x_line = np.logspace(log_start, log_end, num=100)
x_log_line = np.log10(x_line)
y_line = specific_binding(x_line, bmax, kd)

x_points = np.logspace(log_start, log_end, num=16)
x_log_points = np.log10(x_points)
y_points = specific_binding(x_points, bmax, kd)

# set up source data and plot lines that will vary
line_source = ColumnDataSource(data=dict(x=x_line, y=y_line, x_log=x_log_line))
point_source = ColumnDataSource(data=dict(x=x_points, y=y_points, x_log=x_log_points))

plot = figure(x_range=(-9.1, -2.9), y_range=(-10, 210), plot_width=600, plot_height=400, 
              x_axis_label='log[compound (M)]',
              y_axis_label='Specific Binding',
              title='Specific Binding')

plot.line('x_log', 'y', source=line_source, line_width=3, line_alpha=0.6, color='black')
plot.circle('x_log', 'y', source=point_source, size=10, color='black')

# set up static line and annotations
plot.line(x_log_line, y_line, line_width=5, color='blue', line_alpha=0.3)
plot.circle(x_log_points, y_points, size=10, color='blue', line_alpha=0.3)

mytext = Label(x=-5.2, y=110, text='log(Kd) = -6, Bmax = 100', 
               text_color="blue", text_alpha=0.5)
plot.add_layout(mytext)

# add axes lines
vline = Span(location=0, dimension='height', line_color='black', line_width=1, line_alpha=0.3)
hline = Span(location=0, dimension='width', line_color='black', line_width=1, line_alpha=0.3)
plot.renderers.extend([vline, hline])

# set up java script callback function to make plot interactive
bmax_slider = Slider(start=0, end=200, value=100, step=10, title="Bmax")
pkd_slider = Slider(start=-8, end=-3, value=-6, step=0.1, title="log[Kd (M)]")

callback = CustomJS(args=dict(LineSource=line_source, 
                              PointSource=point_source,
                              bmax=bmax_slider, 
                              kd=pkd_slider),
                    code="""
    const LineData = LineSource.data;
    const PointData = PointSource.data;
    const BMAX = bmax.value;
    const KD = Math.pow(10, kd.value);
    const lx = LineData.x;
    const ly = LineData.y;
    const px = PointData.x;
    const py = PointData.y;
    
    // define function(s) for editing data
    function sb(x, BMAX, KD){
        return (x*BMAX)/(x+KD);
    }
    
    // loop over data and edit
    for (var i = 0; i < lx.length; i++) {
        ly[i] = sb(lx[i], BMAX, KD);
    }
    
    for (var i = 0; i < px.length; i++) {
    py[i] = sb(px[i], BMAX, KD);
    }

    // emit changes
    LineSource.change.emit();
    PointSource.change.emit();
""")

# add sliders to plot and display
bmax_slider.js_on_change('value', callback)
pkd_slider.js_on_change('value', callback)

layout = row(
    plot,
    column(bmax_slider, pkd_slider),
)

#output_file("mm.html", title="mm.py example")
show(layout)

## troubleshoot scatchard

In [ ]:
pw = 500
ph = 400

log_start = -9
log_end = -3
bmax= 100
kd = 1e-6

x_line = np.logspace(log_start, log_end, num=100)
x_log_line = np.log10(x_line)
y_line = specific_binding(x_line, bmax, kd)

x_points = np.logspace(log_start, log_end, num=16)
x_log_points = np.log10(x_points)
y_points = specific_binding(x_points, bmax, kd)

b_l, bf_l = scatchard(x_line, bmax, kd)
b_p, bf_p = scatchard(x_points, bmax, kd)

sc_line_source = ColumnDataSource(data=dict(x=x_line, b=b_l, bf=bf_l))
sc_point_source = ColumnDataSource(data=dict(x=x_points, b=b_p, bf=bf_p))

sc_plot = figure(plot_width=pw, plot_height=ph, 
              x_axis_label='Specific Binding',
              y_axis_label='Specific Binding/Free Lignad',
              title='Scatchard Transformation')

sc_plot.line('b', 'bf', source=sc_line_source, line_width=3, line_alpha=0.6, color='black')
sc_plot.circle('b', 'bf', source=sc_point_source, size=10, color='black')

# set up static line and annotations
sc_plot.line(b_l, bf_l, line_width=5, color='blue', line_alpha=0.3)
sc_plot.circle(b_p, bf_p, size=10, color='blue', line_alpha=0.3)

"""mytext = Label(x=-5.2, y=110, text='log(Kd) = -6, Bmax = 100', 
               text_color="blue", text_alpha=0.5)
sc_plot.add_layout(mytext)"""

# add axes lines
vline = Span(location=0, dimension='height', line_color='black', line_width=1, line_alpha=0.3)
hline = Span(location=0, dimension='width', line_color='black', line_width=1, line_alpha=0.3)
sc_plot.renderers.extend([vline, hline])

# make interactive
bmax_slider = Slider(start=0, end=200, value=100, step=10, title="Bmax")
pkd_slider = Slider(start=-8, end=-3, value=-6, step=0.1, title="log[Kd (M)]")

callback = CustomJS(args=dict(LineSource=sc_line_source, 
                              PointSource=sc_point_source,
                              bmax=bmax_slider, 
                              kd=pkd_slider),
                    code="""
    const LineData = LineSource.data;
    const PointData = PointSource.data;
    const BMAX = bmax.value;
    const KD = Math.pow(10, kd.value);
    const lx = LineData.x;
    const lb = LineData.b;
    const lbf = LineData.bf;
    const px = PointData.x;
    const pb = PointData.b;
    const pbf = PointData.bf;
    
    // define function(s) for editing data
    function sb(x, BMAX, KD){
        return (x*BMAX)/(x+KD);
    }
    
    function sc(x, BMAX, KD){
        return sb(x, BMAX, KD)/x;
    }
    
    // loop over data and edit
    for (var i = 0; i < lx.length; i++) {
        lb[i] = sb(lx[i], BMAX, KD);
        lbf[i] = sc(lx[i], BMAX, KD);
    }
    
    for (var i = 0; i < px.length; i++) {
        pb[i] = sb(px[i], BMAX, KD);
        pbf[i] = sc(px[i], BMAX, KD);
    }

    // emit changes
    LineSource.change.emit();
    PointSource.change.emit();
""")

# add sliders to plot and display
bmax_slider.js_on_change('value', callback)
pkd_slider.js_on_change('value', callback)

layout = row(
    sc_plot,
    column(bmax_slider, pkd_slider)
)

#output_file("mm.html", title="mm.py example")
show(layout)

## getting both plots interactive
combine everything into a single line source

In [ ]:
# -------------------------------------------------
# set common parameters to both plots
# -------------------------------------------------
# set plot parameters
pw = 500
ph = 400

# set shared data values
log_start = -9
log_end = -3
bmax= 100
kd = 1e-6

x_line = np.logspace(log_start, log_end, num=100)
x_log_line = np.log10(x_line)
y_line = rec.specific_binding(x_line, bmax, kd)

x_points = np.logspace(log_start, log_end, num=16)
x_log_points = np.log10(x_points)
y_points = rec.specific_binding(x_points, bmax, kd)

b_l, bf_l = rec.scatchard(x_line, bmax, kd)
b_p, bf_p = rec.scatchard(x_points, bmax, kd)

# -------------------------------------------------
# make specific binding plot
# -------------------------------------------------
# set up source data and plot lines that will vary
line_source = ColumnDataSource(data=dict(x=x_line, y=y_line, x_log=x_log_line,
                                             b=b_l, bf=bf_l))
point_source = ColumnDataSource(data=dict(x=x_points, y=y_points, x_log=x_log_points,
                                             b=b_p, bf=bf_p))

sb_plot = figure(plot_width=pw, plot_height=ph, 
              x_axis_label='log[Free Compound (M)]',
              y_axis_label='Specific Binding',
              title='Specific Binding')

sb_plot.line('x_log', 'y', source=line_source, line_width=3, line_alpha=0.6, color='black')
sb_plot.circle('x_log', 'y', source=point_source, size=10, color='black')

# set up static line and annotations
sb_plot.line(x_log_line, y_line, line_width=5, color='blue', line_alpha=0.3)
sb_plot.circle(x_log_points, y_points, size=10, color='blue', line_alpha=0.3)

mytext = Label(x=-5.2, y=110, text='log(Kd) = -6, Bmax = 100', 
               text_color="blue", text_alpha=0.5)
sb_plot.add_layout(mytext)

# add axes lines
vline = Span(location=0, dimension='height', line_color='black', line_width=1, line_alpha=0.3)
hline = Span(location=0, dimension='width', line_color='black', line_width=1, line_alpha=0.3)
sb_plot.renderers.extend([vline, hline])

# -------------------------------------------------
# make scatchard transformation plot
# -------------------------------------------------
sc_plot = figure(plot_width=pw, plot_height=ph, 
              x_axis_label='Specific Binding',
              y_axis_label='Specific Binding/Free Lignad',
              title='Scatchard Transformation')

sc_plot.line('b', 'bf', source=line_source, line_width=3, line_alpha=0.6, color='black')
sc_plot.circle('b', 'bf', source=point_source, size=10, color='black')

# set up static line and annotations
sc_plot.line(b_l, bf_l, line_width=5, color='blue', line_alpha=0.3)
sc_plot.circle(b_p, bf_p, size=10, color='blue', line_alpha=0.3)

mytext = Label(x=50, y=6e7, text='log(Kd) = -6, Bmax = 100', 
               text_color="blue", text_alpha=0.5)
sc_plot.add_layout(mytext)

# add axes lines
vline = Span(location=0, dimension='height', line_color='black', line_width=1, line_alpha=0.3)
hline = Span(location=0, dimension='width', line_color='black', line_width=1, line_alpha=0.3)
sc_plot.renderers.extend([vline, hline])

# -------------------------------------------------
# make plot interactive
# -------------------------------------------------
# set up java script callback function to make plot interactive
bmax_slider = Slider(start=0, end=200, value=100, step=10, title="Bmax")
pkd_slider = Slider(start=-8, end=-3, value=-6, step=0.1, title="log[Kd (M)]")

callback = CustomJS(args=dict(LineSource=line_source, 
                              PointSource=point_source,
                              bmax=bmax_slider, 
                              kd=pkd_slider),
                    code="""
    const LineData = LineSource.data;
    const PointData = PointSource.data;
    const BMAX = bmax.value;
    const KD = Math.pow(10, kd.value);
    const lx = LineData.x;
    const ly = LineData.y;
    const lb = LineData.b;
    const lbf = LineData.bf;
    const px = PointData.x;
    const py = PointData.y;
    const pb = PointData.b;
    const pbf = PointData.bf;
    
    // define function(s) for editing data
    function sb(x, BMAX, KD){
        return (x*BMAX)/(x+KD);
    }
    
    function sc(x, BMAX, KD){
        return sb(x, BMAX, KD)/x;
    }
    
    // loop over data and edit
    for (var i = 0; i < lx.length; i++) {
        ly[i] = sb(lx[i], BMAX, KD);
        lb[i] = sb(lx[i], BMAX, KD);
        lbf[i] = sc(lx[i], BMAX, KD);
    }
    
    for (var i = 0; i < px.length; i++) {
        py[i] = sb(px[i], BMAX, KD);
        pb[i] = sb(px[i], BMAX, KD);
        pbf[i] = sc(px[i], BMAX, KD);
    }

    // emit changes
    LineSource.change.emit();
    PointSource.change.emit();
""")

# add sliders to plot and display
bmax_slider.js_on_change('value', callback)
pkd_slider.js_on_change('value', callback)

# -------------------------------------------------
# set layout and display
# -------------------------------------------------
layout = row(
    sb_plot,
    sc_plot,
    column(bmax_slider,pkd_slider),
)

#output_file("mm.html", title="mm.py example")
show(layout)

## specific binding + hill slope single plot

In [ ]:
## generate bokeh plot using the following data
log_start = -9
log_end = -3
bmax= 100
kd = 1e-6
hill = 1

x_line = np.logspace(log_start, log_end, num=100)
x_log_line = np.log10(x_line)
y_line = rec.specific_binding_hill(x_line, bmax, kd, hill)

x_points = np.logspace(log_start, log_end, num=16)
x_log_points = np.log10(x_points)
y_points = rec.specific_binding_hill(x_points, bmax, kd, hill)

# set up source data and plot lines that will vary
line_source = ColumnDataSource(data=dict(x=x_line, y=y_line, x_log=x_log_line))
point_source = ColumnDataSource(data=dict(x=x_points, y=y_points, x_log=x_log_points))

plot = figure(x_range=(-9.1, -2.9), y_range=(-10, 210), plot_width=600, plot_height=400, 
              x_axis_label='log[compound (M)]',
              y_axis_label='Specific Binding',
              title='Specific Binding')

plot.line('x_log', 'y', source=line_source, line_width=3, line_alpha=0.6, color='black')
plot.circle('x_log', 'y', source=point_source, size=10, color='black')

# set up static line and annotations
plot.line(x_log_line, y_line, line_width=5, color='blue', line_alpha=0.3)
plot.circle(x_log_points, y_points, size=10, color='blue', line_alpha=0.3)

mytext = Label(x=-5.2, y=110, text='log(Kd) = -6, Bmax = 100', 
               text_color="blue", text_alpha=0.5)
plot.add_layout(mytext)

# add axes lines
vline = Span(location=0, dimension='height', line_color='black', line_width=1, line_alpha=0.3)
hline = Span(location=0, dimension='width', line_color='black', line_width=1, line_alpha=0.3)
plot.renderers.extend([vline, hline])

# set up java script callback function to make plot interactive
bmax_slider = Slider(start=0, end=200, value=100, step=10, title="Bmax")
pkd_slider = Slider(start=-8, end=-3, value=-6, step=0.1, title="log[Kd (M)]")
hill_slider = Slider(start=0.1, end=4, value=1, step=0.1, title="Hill Coefficient]")

callback = CustomJS(args=dict(LineSource=line_source, 
                              PointSource=point_source,
                              bmax=bmax_slider, 
                              kd=pkd_slider,
                              hill=hill_slider),
                    code="""
    const LineData = LineSource.data;
    const PointData = PointSource.data;
    const BMAX = bmax.value;
    const KD = Math.pow(10, kd.value);
    const HILL = hill.value;
    const lx = LineData.x;
    const ly = LineData.y;
    const px = PointData.x;
    const py = PointData.y;
    
    // define function(s) for editing data
    function sb(x, BMAX, KD, HILL){
        return (Math.pow(x, HILL)*BMAX)/(Math.pow(x, HILL)+ Math.pow(KD, HILL));
    }
    
    // loop over data and edit
    for (var i = 0; i < lx.length; i++) {
        ly[i] = sb(lx[i], BMAX, KD, HILL);
    }
    
    for (var i = 0; i < px.length; i++) {
    py[i] = sb(px[i], BMAX, KD, HILL);
    }

    // emit changes
    LineSource.change.emit();
    PointSource.change.emit();
""")

# add sliders to plot and display
bmax_slider.js_on_change('value', callback)
pkd_slider.js_on_change('value', callback)
hill_slider.js_on_change('value', callback)

layout = row(
    plot,
    column(bmax_slider, pkd_slider, hill_slider),
)

#output_file("mm.html", title="mm.py example")
show(layout)